# Assignment 5: Pratical Machine Learning

## Overview

Imagine you want to build a classification model for your company. It's not uncommon that you may encounter the following three problems:

* There is no training data available; data labeling may cost you a lot of money and time.
* The number of features is very large, which may not only lead to long training time but also hurt model generalization. 
* There are a number of parameters associated with your model, and you do not know what parameters should be used. 

In this assignment, you will learn how to use Active Learning to address the first problem, learn how to apply a feature selection approach as well as Principal Component Analysis (PCA) to addressing the second one, and learn how to use the Spark ML pipeline API to tune parameters.


After completing this assignment, you should be able to answer the following questions:

**Data Labeling**
1. Why Active Learning?
2. How to implement uncertain sampling, a popular query strategy for Active Learning?
3. How to solve an ER problem using Active Learning?

**Feature Selection**
1. Why Feature Selection?
2. What are typical Feature Selection approaches?
3. How does a filter-based feature selection approach work?
4. How does PCA work?
5. What're the advantages and disadvantages of PCA? 

**Parameter Tuning**
1. Why parameter tuning?
2. What is cross-validation?
3. What is areaUnderROC?
4. How to tune parameters?




Note: In this assignment, the example code was writtin for Python 2. If you use Python 3, please feel free to make changes (e.g., print x --> print(x)).

You can download the datasets of each assignment from http://tiny.cc/cmpt733-datasets.

## Part 1. Data Labeling

[Active learning](http://tiny.cc/al-wiki) is a certain type of ML algorithms that can train a high-quality ML model with small data-labeling cost. Its basic idea is quite easy to understand. Consider a typical supervised ML problem, which requires a (relatively) large training dataset. In the training dataset, there may be only a small number of data points that are beneficial to the trained ML model. In other words, labeling the small number of data points is enough to train a high-quality ML model. The goal of active learning is to help us to identify the data points. 


In this assignment, we will revisit [Entity Resolution](https://sfu-db.github.io/bigdata-cmpt733/Lectures/lec2.pdf) and develop an Active Learning approach for it. Recall that entity Resolution (ER) is defined as finding different records that refer to the same real-world entity, e.g., "iPhone 4-th Generation" vs. "iPhone Four". It is central to data integration and cleaning. The following figure shows the architecture of an entity resolution solution. It consists of four major steps. **I will provide you the source code for Steps 1, 2, 4. Your job is to implement Step 3.**  

<img src="img/arch.png", width=800/>

### Step 1. Read Data

The [restaurant data](restaurant.csv) is in an CSV file. Here is the code to read it. 

In [125]:
import pandas as pd

df = pd.read_csv('restaurant.csv')
data = df.values.tolist()
# print(data)
print("(#Rows, #Cols) :", df.shape)
df.head(5)

(#Rows, #Cols) : (858, 5)


,id,name,address,city,type
0,1,arnie morton's of chicago,435 s. la cienega blv.,los angeles,american
1,2,arnie morton's of chicago,435 s. la cienega blvd.,los angeles,steakhouses
2,3,art's delicatessen,12224 ventura blvd.,studio city,american
3,4,art's deli,12224 ventura blvd.,studio city,delis
4,5,hotel bel-air,701 stone canyon rd.,bel air,californian


From this dataset, you will find many matching record pairs. For example, the first two rows shown above are matching (i.e., refer to the same real-world entity). You can check out all matching record pairs in the [true_matches.json](true_matches.json) file. 

### Step 2. Remove Obviously Non-matching Pairs

A naive implementation of entity resolution is to enumerate all $n^2$ pairs and check whether they are matching or not. As you've learnt in [Assignment 2](https://rawgit.com/sfu-db/bigdata-cmpt733/master/Assignments/A2/A2.html), we can avoid $n^2$ comparisons using a similarity-join algorithm. 

Here is the code. After running the code, you will get 678 similar pairs ordered by their similarity decreasingly.

In [126]:
from a9_utils import *

data = df.values.tolist()
simpairs = simjoin(data)

print("Num of Pairs: ", len(data)*(len(data)-1)/2)
print("Num of Similar Pairs: ", len(simpairs))
print("The Most Similar Pair: ", simpairs[0])

Num of Pairs:  367653.0
Num of Similar Pairs:  678
The Most Similar Pair:  ([170, "mary mac's tea room", '224 ponce de leon ave.', 'atlanta', 'southern/soul'], [169, "mary mac's tea room", '224 ponce de leon ave.', 'atlanta', 'southern'])


We can see that `simjoin` helps us remove the number of pairs from 367653 to 678. But, there are still many non-matching pairs in `simpairs` (see below). 

In [127]:
print(simpairs[-1])

([764, "buzio's in the rio", '3700 w. flamingo rd.', 'las vegas', 'seafood'], [542, 'carnival world', '3700 w. flamingo rd.', 'las vegas', 'buffets'])


Next, we will use active learning to train a classifier, and then use the classifier to classify each pair in `simpairs` as either "matching" or "nonmatching". 

### Step 3. Active Learning (Task A)

Given a set of similar pairs, what you need to do next is to iteratively train a classifier to decide which pairs are truly matching. We are going to use [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression) as our classifier. 

#### Featurization

To train the model, the first thing you need to think about is how to featurize data. That is, transforming each similar pair to a feature vector. Please use the `featurize()` function for featurization. The function takes a pair as input and returns a vector of 6 features.

In [128]:
from a9_utils import featurize

print("The feature vector of the first pair: ", featurize(simpairs[0]))

The feature vector of the first pair:  (1.0, 1.0, 1.0, 1.0, 1.0, 0.6153846153846154)


#### Initialization

At the beginning, all the pairs are unlabeled. To initialize a model, we first pick up ten pairs and then label each pair using  the `crowdsourcing()` function. You can assume that `crowdsourcing()` will ask a crowd worker (e.g., on Amazon Mechanical Turk) to label a pair. 


`crowdsourcing(pair)` is a function that simulates the use of crowdsourcing to label a pair
  
  - **Input:**	pair – A pair of records 

  - **Output:**	Boolean –  *True*: The pair of records are matching; *False*: The pair of records are NOT matching;

Please use the following code to do the initialization. 

In [129]:
from a9_utils import crowdsourcing

# choose the most/least similar five pairs as initial training data
init_pairs = simpairs[:5] + simpairs[-5:]
matches = []
nonmatches = []
target_matches = []
target_nonmatches = []

for pair in init_pairs:
    is_match = crowdsourcing(pair)
    if is_match == True:
        matches.append(pair)
        target_matches.append(is_match)
    else:
        nonmatches.append(pair)
        target_nonmatches.append(is_match)
        
print("Number of matches: ", len(matches))
print("Number of nonmatches: ", len(nonmatches))

Are they matching?
[170, "mary mac's tea room", '224 ponce de leon ave.', 'atlanta', 'southern/soul']
[169, "mary mac's tea room", '224 ponce de leon ave.', 'atlanta', 'southern']
Answer: Yes
Are they matching?
[88, 'manhattan ocean club', '57 w. 58th st.', 'new york city', 'seafood']
[87, 'manhattan ocean club', '57 w. 58th st.', 'new york', 'seafood']
Answer: Yes
Are they matching?
[112, 'san domenico', '240 central park s.', 'new york city', 'italian']
[111, 'san domenico', '240 central park s', 'new york', 'italian']
Answer: Yes
Are they matching?
[197, 'fleur de lys', '777 sutter st.', 'san francisco', 'french (new)']
[196, 'fleur de lys', '777 sutter st.', 'san francisco', 'french']
Answer: Yes
Are they matching?
[8, 'cafe bizou', '14016 ventura blvd.', 'sherman oaks', 'french bistro']
[7, 'cafe bizou', '14016 ventura blvd.', 'sherman oaks', 'french']
Answer: Yes
Are they matching?
[709, 'arcadia', '21 e. 62nd st.', 'new york city', 'american (new)']
[66, 'four seasons', '99 e. 5

#### Interactive Model Training

Here is the only code you need to write in Part 1.


In [130]:
from a9_utils import featurize, crowdsourcing
from sklearn.linear_model import LogisticRegression
import numpy as np

labeled_pairs = matches + nonmatches
target = target_matches + target_nonmatches
unlabeled_pairs = [p for p in simpairs if p not in labeled_pairs]
iter_num = 5
feature = []
feature_unlabelled = []
len_labpairs = len(labeled_pairs)
len_unlabpairs = len(unlabeled_pairs)
threshold = 0.5

# Featurize the labeled points so that it can be fed to the model.
for iter_var in range(len_labpairs):
    feature.append(featurize(labeled_pairs[iter_var]))

# Featurize the unlabeled points so that it can be fed to the model.
for iter_var in range(len_unlabpairs):
    feature_unlabelled.append(featurize(unlabeled_pairs[iter_var]))

# Logistic regression initialized on a random set of labeled 10 points picked earlier.
model = LogisticRegression()
model.fit(feature, target)
print("Model succesfully Initialized/trained using labeled pairs.")
difference_pairs = []

# Iteractive modelling for a max of 5 iterations.
for iter_var in range(iter_num):
    print("\n\nModel predicting probabilities for unlabelled pairs. Iteration: ", iter_var+1)
    difference_pairs = []
    for pair in feature_unlabelled:
        probability = model.predict_proba(np.array(pair).reshape(1, -1))
        # Store distance between the predicted probabilities for both classes for each point
        difference_pairs.append(abs(probability[0][0]-probability[0][1]))

    # Find the most uncertain point of the unlabelled points, maintaining a threshold of 0.5 to take care of false positives
    min_diff = min(difference_pairs)
    index_uncpair = difference_pairs.index(min_diff)
    if len([index_uncpair]) > 1:
        index_uncpair = index_uncpair[0]
    else:
        index_uncpair = index_uncpair
#     print(index_uncpair)

    print("\nLeast difference between the predictions for both matching and non-matching class of an unlabeled point: ", min_diff)
    uncertain_pair = unlabeled_pairs[index_uncpair]
    print("\nMost uncertain pair: ", uncertain_pair)
    
    # Append the new labeled point to labeled_pairs and remove it from unlabeled_pairs.
    labeled_pairs.append(uncertain_pair)
    unlabeled_pairs.remove(uncertain_pair)

    feature_label = featurize(uncertain_pair)
    feature.append(feature_label)
    target_label = crowdsourcing(uncertain_pair)
    target.append(target_label)
    
    # Train classifier again based on updated labeled pairs information.
    model.fit(feature, target)

Model succesfully Initialized/trained using labeled pairs.


Model predicting probabilities for unlabelled pairs. Iteration:  1

Least difference between the predictions for both matching and non-matching class of an unlabeled point:  0.0067161235987684975

Most uncertain pair:  ([741, 'palm too', '840 second ave.', 'new york city', 'steakhouses'], [740, 'palm', '837 second ave.', 'new york city', 'steakhouses'])
Are they matching?
[741, 'palm too', '840 second ave.', 'new york city', 'steakhouses']
[740, 'palm', '837 second ave.', 'new york city', 'steakhouses']
Answer: No


Model predicting probabilities for unlabelled pairs. Iteration:  2

Least difference between the predictions for both matching and non-matching class of an unlabeled point:  0.009980589260801143

Most uncertain pair:  ([364, 'global', '33 93 2nd ave.  between 5th and 6th sts.', 'new york', 'american'], [350, 'first', '87 1st ave.  between 5th and 6th sts.', 'new york', 'american'])
Are they matching?
[364, 'global

**[Algorithm Description].**   Active learning has many [query strategies](http://tiny.cc/al-wiki-qs) to decide which data point should be labeled. You need to implement uncertain sampling for Task A. The algorithm trains an initial model on `labeled_pairs`. Then, it iteratively trains a model. At each iteration, it first applies the model to `unlabeled_pairs`, and makes a prediction on each unlabeled pair along with a probability, where the probability indicates the confidence of the prediction. After that, it selects the most uncertain pair (If there is still a tie, break it randomly),  and call the `crowdsroucing()` function to label the pair. After the pair is labeled, it updates `labeled_pairs` and `unlabeled_pairs`, and then retrain the model on `labeled_pairs`.

**[Input].** 
- `labeled_pairs`: 10 labeled pairs (by default)
- `unlabeled_pairs`: 668 unlabeled pairs (by default)
- `iter_num`: 5 (by default)

**[Output].** 
- `model`: A logistic regression model built by scikit-learn


### Step 4. Model Evaluation

[Precision, Recall](https://en.wikipedia.org/wiki/Precision_and_recall), [F-Score](https://en.wikipedia.org/wiki/F1_score) are commonly used to evaluate an entity-resolution result. After training an model, you can use the following code to evalute it.

In [131]:
import json
from a9_utils import evaluate
import numpy as np
     
sp_features = np.array([featurize(sp) for sp in simpairs])
label = model.predict(sp_features)
pair_label = zip(simpairs, label)

identified_matches = []
for pair, label in pair_label:
    if label == 1:
        identified_matches.append(pair)
        
precision, recall, fscore = evaluate(identified_matches)

print("Precision:", precision)
print("Recall:", recall)
print("Fscore:", fscore)
   

Precision: 0.9150943396226415
Recall: 0.9150943396226415
Fscore: 0.9150943396226415


## Part 2. Feature Selection

Feature selection and dimensionality reduction have been extensively studied in the ML community. In this part, I designed two simple tasks for you to get familiar with these topics. If you want to know more, please refer to Prof. Michael Jordan's [Practical Machine Learning](https://people.eecs.berkeley.edu/~jordan/courses/294-fall09/) course at UC Berkeley. 

### Task B. Filter-based Feature Selection

There are [three classes](https://en.wikipedia.org/wiki/Feature_selection#Main_principles) of feature-selection methods: filter-based, wrapper-based, and embeded-based. Filter-based is the most simple one. Its basic idea is to assign a heuristic score to each feature to filter out the "obviously" useless ones. There are many ways to compute the score, e.g., chi-squared, information gain, correlation, and mutual information. We use [chi-squared](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html) for Task B. 

Imagine you have a collection of newsgroup documents, and you want to build a classification model to predicate the topic of each newsgroup document: "comp.graphics (0)" or "sci.space (1)". You use bag of words to represent each document. That is, each feature is a single word. I have already helped you load the dataset and generated the feature vectors (see the code below). 

In [132]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer


data_train = fetch_20newsgroups(subset='train', categories=['comp.graphics','sci.space'],
                                shuffle=True, random_state=42,
                                remove=('headers', 'footers', 'quotes'))

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data_train.data)
y = data_train.target

# print(X)
# print(y)

print("Target Labels:", data_train.target_names)
print("(NSamples, NFeatures):", X.shape)


Target Labels: ['comp.graphics', 'sci.space']
(NSamples, NFeatures): (1177, 19493)


From the output of the above code, you find that the number of features (19493) is very large, so you want to use a filter-based method to choose the top-10 words as your features. 

Please write your code below. The code computes chi-squared stat between each feature and the target label, and return the top-10 words with the highest score. 

In [133]:
from sklearn.feature_selection import SelectKBest, chi2

chi10 = SelectKBest(chi2, k=10)
X_train = chi10.fit_transform(X, y)
feature_names = vectorizer.get_feature_names()

# Get top 10 features
if feature_names:
    feature_names = [feature_names[iter_var] for iter_var in ch2.get_support(indices=True)]
    print(feature_names)
    
print('\nTop 10 chosen features based on highest chi-squared value/score')

for i in range(len(feature_names)):
    print('Feature ', i+1, ': ', feature_names[i])

['3d', 'file', 'files', 'graphics', 'image', 'moon', 'nasa', 'orbit', 'space', 'windows']

Top 10 chosen features based on highest chi-squared value/score
Feature  1 :  3d
Feature  2 :  file
Feature  3 :  files
Feature  4 :  graphics
Feature  5 :  image
Feature  6 :  moon
Feature  7 :  nasa
Feature  8 :  orbit
Feature  9 :  space
Feature  10 :  windows


### Task C. Principal Component Analysis (PCA)

PCA is a well-known dimensionality reduction algorithm. Obviously, we can use PCA to transform each feature vector to a 10-D feature vector. See the code below. 

In [134]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
pca.fit(X.toarray())
print(pca.fit(X.toarray()))
print(pca.explained_variance_ratio_)

PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)
[0.01690229 0.00799231 0.00645212 0.00520741 0.00477428 0.00446403
 0.00415196 0.00391931 0.00384415 0.00367162]


Both filter-based feature selection and PCA can help us reduce the number of features in a feature vector. The question is what's the difference between them. In this task, you don't need to write any code. Just let me know what you think about the advantages and disadvantages of PCA compared to filter-based feature section. Please list (at least) two advantages and two disadvantages. 

**Advantages of PCA**
 - PCA can reduce the amount of dimensions without losing much information, when patterns in the data are found and compressed.
 - PCA is a powerful tool that can analyze large datasets with ease. It can easily handle huge volumes of data.
   

**Disadvantages of PCA**
- PCA assumes that the principle components are a linear combination of the original features. If this is not true, PCA will not give sensible results.
- PCA further assumes that the principle components are orthogonal.If this is not true, PCA will not give sensible results.

## Part 3. Parameter Tuning

### Preliminary

Imagine you have a collection of newsgroup documents, and you want to build a classification model to predicate the topic of each newsgroup document: "science (1)" or "non-science (0)". Here is the Spark program (below) that can help you to finish the job. ** Your preliminary task is to read the code and understand how it works. ** I highly recommend you to read through the [Spark ML Pipeline Programming Guide](http://spark.apache.org/docs/latest/ml-pipeline.html), which is very well written, and will aid your understanding of the code. 

In [135]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# sc = SparkContext('local')
spark = SparkSession(sc)

trainDF = spark.read.parquet("20news_train.parquet")

# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features", numFeatures=1000)
lr = LogisticRegression(maxIter=20, regParam=0.1)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training data.
model = pipeline.fit(trainDF)

# Evaluate the model on testing data
testDF = spark.read.parquet("20news_test.parquet")
prediction = model.transform(testDF)
evaluator = BinaryClassificationEvaluator()
print(evaluator.evaluate(prediction))


0.7486382404196998


If you run the program, you will see that the trained model can only get an [areaUnderROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve) of ~0.75 on the testing dataset.

### Task D - Parameter Tuning

Now it comes to the final task of the assignment. Let's take a look at the above program. It actually used `numFeatures=1000` and `regParam=0.1` to train the model. One natural question is that if we used different values for the two parameters, would that lead to a better model (i.e., a higher areaUnderROC)?


In this task, your job is to complete the following code. 

In [136]:
# The code will tune the following two parameters on the training dataset.
#
# numFeatures[1000, 5000, 10000] and regParam[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#
# and will use 2-fold cross-validation for model evaluation. 

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Create a paramgrid for hyper paramater tuning with different values given for numfeatures and regparam.
tune_params = ParamGridBuilder().addGrid(hashingTF.numFeatures, [1000, 5000, 10000]).addGrid(lr.regParam, [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).build()

# Create a cross validator that uses the paramgrid parameters and uses 2-fold cross validation.
validate_crossval = CrossValidator(estimator=pipeline, estimatorParamMaps=tune_params, evaluator=BinaryClassificationEvaluator(), numFolds=2)  

# Fit the new model to the traning set and make predcitions on test set, evaluate the same.
model = validate_crossval.fit(trainDF)
predicted_val = model.transform(testDF)
evaluator = BinaryClassificationEvaluator()
value = evaluator.evaluate(predicted_val)
print('Evaluated performance: ', value)

Evaluated performance:  0.8536581733181879


Once you get the best model with the best parameters, please apply the model to the testing dataset, and `print` the new areaUnderROC value. 

## Submission

Download [A5.zip](A5.zip). Complete Tasks A, B, C and D in A5.ipynb, and submit it to the CourSys activity Assignment 5.